In [31]:
import sys

sys.path.append("..")

In [32]:
from src import utils, preprocessing

In [33]:
# Initialize subject and session information
subject = "sub-001"
sessions = ["ses-001", "ses-002", "ses-003", "ses-004"]

In [34]:
# If subject is sub-001 or sub-006, load the mne-raw objects from the realigned directory
if subject in ["sub-001", "sub-006"]:
    directory = f"../data/interim/{subject}/eeg-realigned"
else:
    directory = f"../data/interim/{subject}/eeg"

# Load mne-raw objects to the notebook
raw_list = []
for session in sessions:
    filename = f"{subject}_task-rest_eeg_{session}_raw.fif"
    raw = utils.load_mne_data(directory, filename)
    raw_list.append(raw)

In [35]:
# Apply preprocessing steps to each mne-raw object in raw_list
preprocessed_raw_list = []
for raw in raw_list:
    # Interpolate NaNs values
    data = raw.get_data()
    data_interpolated = preprocessing.interpolate_nans(data)
    raw._data = data_interpolated

    # Apply preprocessing steps
    raw_filtered = preprocessing.notch_filter(raw, 60)
    raw_filtered = preprocessing.bandpass_filter(raw_filtered, 0.1, 40)
    raw_filtered = preprocessing.re_reference(raw_filtered, "average")
    preprocessed_raw_list.append(raw_filtered)

In [36]:
# Save preprocessed raw data for each session
directory = f"../data/processed/{subject}/eeg"
for i, session in enumerate(sessions):
    filename = f"{subject}_task-rest_eeg_{session}_filtered_raw.fif"
    utils.save_mne_data(preprocessed_raw_list[i], directory, filename)